In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2> Загрузка библиотек 

In [ ]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords   

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

<h2> Загрузка данных 

In [ ]:
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',header=0,delimiter="\t",quoting=3)
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip',header=0,delimiter="\t",quoting=3)

<h2> Преобразование данных

In [ ]:
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review).get_text()     
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return(" ".join(meaningful_words)) 

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    clean_train_reviews.append(review_to_words(train["review"][i]))

In [ ]:
vectorizer = CountVectorizer(analyzer="word",tokenizer=None, preprocessor=None,stop_words=None,max_features=5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
train_data_features.shape

In [ ]:
vocab = vectorizer.get_feature_names()

<h2> Создание и обучение модели

In [ ]:
#используем классификатор случайного леса
rfc = RandomForestClassifier(n_estimators = 100, n_jobs=-1)
rfc.fit(train_data_features, train["sentiment"])

In [ ]:
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = rfc.predict(test_data_features)
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("My_prediction.csv", index=False, quoting=3)